In [7]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
from glob import glob

/tmp/ipykernel_34671/1679044566.py:9: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [8]:
# %load_ext autoreload
# %autoreload 2

from dataset import SoccerDataset
from trace_regressor import *
from trace_discriminator import TraceDiscriminator
from poss_classifier import PossClassifier, PossTransformerClassifier
from utils import train

In [9]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
files = glob('data/metrica_traces/*.csv') + glob('data/final_traces/final_traces/*.csv')
files.sort()
train_files = files[:-1]
val_files = files[-1:]
train_files, val_files

(['data/final_traces/final_traces/12864_12868.csv',
  'data/final_traces/final_traces/12887_12911.csv',
  'data/final_traces/final_traces/13044_13055.csv',
  'data/final_traces/final_traces/13213_13216.csv',
  'data/final_traces/final_traces/13224_13278.csv',
  'data/final_traces/final_traces/13471_13479.csv',
  'data/final_traces/final_traces/14082_14088.csv',
  'data/final_traces/final_traces/14212_14256.csv',
  'data/final_traces/final_traces/14247_14274.csv',
  'data/final_traces/final_traces/14249_14258.csv',
  'data/final_traces/final_traces/14524_14534.csv',
  'data/final_traces/final_traces/14572_14643.csv',
  'data/final_traces/final_traces/14890_14929.csv',
  'data/final_traces/final_traces/15215_15218.csv',
  'data/final_traces/final_traces/15216_15217.csv',
  'data/final_traces/final_traces/15569_15580.csv',
  'data/final_traces/final_traces/15571_15581.csv',
  'data/final_traces/final_traces/15849_15855.csv',
  'data/final_traces/final_traces/15860_15884.csv',
  'data/fina

### Training a Ball Regressor

In [11]:
train_dataset = SoccerDataset(train_files, mode='true_gk', poss_encode=True)
val_dataset = SoccerDataset(val_files, mode='true_gk', poss_encode=True)

In [7]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [15]:
target_type = 'gk'
model_type = 'settransformer'
loss_type = 'mse'

if model_type == 'setlstm':
    trace_regressor = TraceSetLSTM(target_type=target_type).to(device)
elif model_type == 'seq2seq':
    trace_regressor = TraceSeq2Seq(target_type=target_type).to(device)
elif model_type == 'settransformer':
    trace_regressor = TraceSetTransformer(target_type=target_type).to(device)

#model_path = f'saved_models/{target_type}_{model_type}_{loss_type}.pt'
#print(trace_regressor.load_state_dict(torch.load(model_path)))
#trace_regressor.train()

In [21]:
# train(trace_regressor, train_loader, val_loader, phys_loss=False, device=device)

### Training a Team Possession Classifier

In [7]:
#train_dataset = SoccerDataset(train_files, mode='true_gk', poss_encode=True)
#val_dataset = SoccerDataset(val_files, mode='true_gk', poss_encode=True)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [8]:
poss_classifier = PossTransformerClassifier(ball_trace_given=False, mode='team', hidden_dim=128).to(device)

In [24]:
#model_path = f'saved_models/team_poss_classifier.pt'
#poss_classifier.load_state_dict(torch.load(model_path))

In [9]:
train(poss_classifier, train_loader, val_loader, phys_loss=False, device=device)

  0%|          | 0/36867 [00:00<?, ?it/s]

/home/toc/anaconda3/envs/balltrack/lib/python3.8/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853042/work/aten/src/ATen/native/TensorCompare.cpp:333.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


cos_num device : cuda:0
cos_denom device : cuda:0
eps_device :  cpu


NameError: name 'asd' is not defined

In [13]:
val_dataset[6000][-1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [ ]:
train_dataset[420][0].shape

In [23]:
torch.save(poss_classifier.state_dict(), 'saved_models/team_poss_classifier_128.pt')

FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/team_poss_classifier_128.pt'